In [2]:
import glob, os
import re

In [3]:
glob_list = []

#loop through subject folders and glob
for subject in range(25):
    glob_list.append(glob.glob(str(subject) + "\\[0-9a-zA-Z]*.csv"))
    
#function to flatten glob
flatten = lambda l: [item for sublist in l for item in sublist]
glob_list = flatten(glob_list)
glob_list

['0\\0_Left.csv',
 '0\\0_Right.csv',
 '0\\10_Left.csv',
 '0\\10_Right.csv',
 '0\\1_Left.csv',
 '0\\1_Right.csv',
 '0\\2_Left.csv',
 '0\\2_Right.csv',
 '0\\3_Left.csv',
 '0\\3_Right.csv',
 '0\\4_Left.csv',
 '0\\4_Right.csv',
 '0\\5_Left.csv',
 '0\\5_Right.csv',
 '0\\6_Left.csv',
 '0\\6_Right.csv',
 '0\\7_Left.csv',
 '0\\7_Right.csv',
 '0\\8_Left.csv',
 '0\\8_Right.csv',
 '0\\9_Left.csv',
 '0\\9_Right.csv',
 '0\\Big_Left.csv',
 '0\\Big_Right.csv',
 '0\\Blue_Left.csv',
 '0\\Blue_Right.csv',
 '0\\Brush_Left.csv',
 '0\\Brush_Right.csv',
 '0\\Bug_Left.csv',
 '0\\Bug_Right.csv',
 '0\\Candy_Left.csv',
 '0\\Candy_Right.csv',
 '0\\CarDrive_Left.csv',
 '0\\CarDrive_Right.csv',
 '0\\Cat_Left.csv',
 '0\\Cat_Right.csv',
 '0\\Cereal_Left.csv',
 '0\\Cereal_Right.csv',
 '0\\Clothes_Left.csv',
 '0\\Clothes_Right.csv',
 '0\\Coat_Left.csv',
 '0\\Coat_Right.csv',
 '0\\Cold_Left.csv',
 '0\\Cold_Right.csv',
 '0\\Come_Left.csv',
 '0\\Come_Right.csv',
 '0\\Cost_Left.csv',
 '0\\Cost_Right.csv',
 '0\\Cry_Left.cs

In [3]:
#make a list of tuples of hand pairings
lr_pairs = list(zip(*[iter(glob_list)]*2))
for i in range(25):
    lr_pairs.remove((str(i)+'\\Blue_Left.csv', str(i)+'\\Blue_Right.csv'))
lr_pairs

[('0\\Big_Left.csv', '0\\Big_Right.csv'),
 ('0\\Brush_Left.csv', '0\\Brush_Right.csv'),
 ('0\\Bug_Left.csv', '0\\Bug_Right.csv'),
 ('0\\Candy_Left.csv', '0\\Candy_Right.csv'),
 ('0\\CarDrive_Left.csv', '0\\CarDrive_Right.csv'),
 ('0\\Cat_Left.csv', '0\\Cat_Right.csv'),
 ('0\\Cereal_Left.csv', '0\\Cereal_Right.csv'),
 ('0\\Clothes_Left.csv', '0\\Clothes_Right.csv'),
 ('0\\Coat_Left.csv', '0\\Coat_Right.csv'),
 ('0\\Cold_Left.csv', '0\\Cold_Right.csv'),
 ('0\\Come_Left.csv', '0\\Come_Right.csv'),
 ('0\\Cost_Left.csv', '0\\Cost_Right.csv'),
 ('0\\Cry_Left.csv', '0\\Cry_Right.csv'),
 ('0\\Dad_Left.csv', '0\\Dad_Right.csv'),
 ('0\\Deaf_Left.csv', '0\\Deaf_Right.csv'),
 ('0\\Dog_Left.csv', '0\\Dog_Right.csv'),
 ('0\\Drink_Left.csv', '0\\Drink_Right.csv'),
 ('0\\Egg_Left.csv', '0\\Egg_Right.csv'),
 ('0\\Finish_Left.csv', '0\\Finish_Right.csv'),
 ('0\\Good_Left.csv', '0\\Good_Right.csv'),
 ('0\\Go_Left.csv', '0\\Go_Right.csv'),
 ('0\\Green_Left.csv', '0\\Green_Right.csv'),
 ('0\\Happy_Left.csv

In [4]:
import pandas as pd

df_list = []

for pair in lr_pairs:
    df_left = pd.read_csv(pair[0], index_col=None).drop(['Unnamed: 0'], axis = 1)
    df_right = pd.read_csv(pair[1], index_col=None).drop(['Unnamed: 0'], axis = 1)
    
    #rename columns
    df_left = df_left.add_prefix('left')
    df_right = df_right.add_prefix('right')
    
    #merge
    df = pd.merge(df_left, df_right, left_on='leftTime', right_on='rightTime').drop('rightTime', axis = 1)

    #covert first column to time object
    df['leftTime'] = pd.to_datetime(df['leftTime'].str[:-3], format = '%H:%M:%S.%f')
    
    #difference between rows
    df = df.diff().iloc[1:]
    df['leftTime'] = df['leftTime'].dt.total_seconds()
    
    #add sign and subject using regex of file name
    subject_sign = re.split(r'\\', re.findall('^[^_]+(?=_)', pair[0])[0])
    df.insert(loc = 0, column = 'Subject', value = subject_sign[0])
    df.insert(loc = 0, column = 'Sign', value = subject_sign[1])
    
#this makes velocity columns

#     for col in df.iloc[:,2:]:
#         df[col + ' velocity'] = df[col]/df['Time']
    
    df_list.append(df)

In [5]:
#all signs (except 'Blue') for 25 subjects
len(df_list)

1225

In [6]:
#example entry
df_list[35]

,Sign,Subject,leftTime,left thumbProximal_L_X,left thumbProximal_L_Y,left thumbProximal_L_Z,left thumbDistal_L_X,left thumbDistal_L_Y,left thumbDistal_L_Z,left thumbEF_L_X,...,right armFrontRadius_L_Z,right armFrontUlna_L_X,right armFrontUlna_L_Y,right armFrontUlna_L_Z,right armBackLateral_L_X,right armBackLateral_L_Y,right armBackLateral_L_Z,right armBackMedial_L_X,right armBackMedial_L_Y,right armBackMedial_L_Z
1,Small,0,0.028,0.000171,-0.000109,0.000052,0.000064,0.000017,0.000009,0.000070,...,-0.000060,0.000145,0.000235,-0.000128,0.000132,0.000590,-0.000350,0.000087,0.000532,-0.000418
2,Small,0,0.030,0.000140,-0.000158,0.000050,-0.000004,-0.000027,0.000020,-0.000063,...,-0.000014,0.000059,-0.000037,-0.000120,0.000500,0.000602,-0.000346,0.000431,0.000513,-0.000452
3,Small,0,0.033,0.000190,-0.000121,0.000254,0.000046,-0.000091,0.000305,-0.000059,...,0.000016,-0.000043,0.000001,-0.000134,0.000565,0.000818,-0.000400,0.000417,0.000694,-0.000550
4,Small,0,0.033,0.000177,-0.000111,0.000324,0.000134,-0.000226,0.000440,0.000051,...,-0.000081,-0.000147,-0.000108,-0.000191,0.000444,0.000430,-0.000291,0.000290,0.000343,-0.000400
5,Small,0,0.029,0.000087,-0.000161,0.000272,0.000101,-0.000310,0.000384,0.000076,...,-0.000171,-0.000157,-0.000158,-0.000165,0.000014,-0.000082,-0.000145,0.000063,-0.000079,-0.000139
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
806,Small,0,0.044,-0.000002,0.000371,-0.000081,0.000136,0.000507,-0.000258,0.000224,...,0.000505,-0.000911,-0.000149,0.000546,-0.000228,-0.000424,0.000759,0.000043,-0.000278,0.000800
807,Small,0,0.030,0.000149,0.000003,-0.000067,0.000241,0.000127,-0.000229,0.000282,...,0.000850,-0.002754,-0.001214,0.000880,-0.000856,-0.000741,0.000764,-0.000970,-0.000761,0.000794
808,Small,0,0.049,0.004451,-0.000390,0.003292,0.005072,-0.000417,0.003268,0.005358,...,0.003047,-0.009288,-0.002040,0.003083,-0.000401,0.000889,0.002432,-0.000858,0.000739,0.002468
809,Small,0,0.042,0.009015,-0.003349,0.004793,0.009673,-0.003652,0.005084,0.010102,...,0.003117,-0.012609,-0.003113,0.003078,-0.001080,0.001319,0.002561,-0.001828,0.001007,0.002522


In [7]:
#concatenate list of frames
sign_frame = pd.concat(df_list, axis=0, ignore_index=False)
sign_frame

,Sign,Subject,leftTime,left thumbProximal_L_X,left thumbProximal_L_Y,left thumbProximal_L_Z,left thumbDistal_L_X,left thumbDistal_L_Y,left thumbDistal_L_Z,left thumbEF_L_X,...,right armFrontRadius_L_Z,right armFrontUlna_L_X,right armFrontUlna_L_Y,right armFrontUlna_L_Z,right armBackLateral_L_X,right armBackLateral_L_Y,right armBackLateral_L_Z,right armBackMedial_L_X,right armBackMedial_L_Y,right armBackMedial_L_Z
1,Big,0,0.034,0.000548,-0.000211,-0.000366,0.000509,0.000025,-0.000463,0.000466,...,-2.264000e-04,0.000170,0.000627,-0.000197,-0.000267,0.000047,0.000104,0.000043,0.000249,0.000133
2,Big,0,0.033,0.000049,0.000050,-0.000382,0.000254,-0.000032,-0.000403,0.000404,...,-4.440000e-05,0.000079,0.000201,-0.000038,-0.000019,0.000047,0.000048,0.000087,0.000113,0.000055
3,Big,0,0.027,-0.000028,0.000140,-0.000342,0.000123,0.000058,-0.000349,0.000269,...,1.332000e-04,0.000027,0.000068,0.000110,0.000020,0.000198,0.000014,0.000106,0.000217,-0.000009
4,Big,0,0.035,-0.000194,0.000160,-0.000154,-0.000177,0.000152,-0.000155,-0.000147,...,1.175000e-04,-0.000029,-0.000080,0.000084,0.000154,0.000287,-0.000132,0.000115,0.000224,-0.000165
5,Big,0,0.035,-0.000172,0.000052,0.000150,-0.000235,0.000053,0.000168,-0.000285,...,2.000000e-07,-0.000094,-0.000100,-0.000016,-0.000002,0.000068,-0.000113,0.000005,0.000052,-0.000130
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
839,Yellow,24,0.041,-0.000081,0.000077,-0.000299,-0.000051,-0.000085,-0.000311,-0.000089,...,7.389000e-04,-0.003499,-0.001501,-0.000228,-0.027895,-0.018280,0.000122,-0.026445,-0.018375,-0.000844
840,Yellow,24,0.040,-0.000211,0.000190,-0.000301,-0.000174,0.000163,-0.000348,-0.000167,...,-2.416600e-03,-0.001886,-0.000467,-0.002440,-0.006380,0.002420,-0.007261,-0.008172,-0.000481,-0.007284
841,Yellow,24,0.041,-0.000124,0.000341,-0.000163,-0.000145,0.000336,-0.000134,-0.000140,...,-1.663600e-03,0.001196,0.001054,-0.001449,0.009235,0.008616,-0.003942,0.007797,0.007248,-0.003728
842,Yellow,24,0.034,-0.000187,0.000359,-0.000166,-0.000138,0.000345,-0.000232,-0.000073,...,-3.342000e-04,0.000911,0.000960,-0.000300,0.000714,0.000893,-0.000346,0.000451,0.000660,-0.000311


In [8]:
agg_df = sign_frame.groupby(['Subject','Sign']).agg(['mean','std'])
agg_df = agg_df.reset_index()
agg_df.head()

Subject      Sign  leftTime           left thumbProximal_L_X            \
                         mean       std                   mean       std   
0       0       Big  0.036579  0.004711              -0.000046  0.006123   
1       0     Brush  0.038698  0.005016              -0.000027  0.001896   
2       0       Bug  0.036683  0.004813               0.000027  0.000477   
3       0     Candy  0.038697  0.005026               0.000017  0.000395   
4       0  CarDrive  0.036469  0.004930              -0.000048  0.004520   

  left thumbProximal_L_Y           left thumbProximal_L_Z            ...  \
                    mean       std                   mean       std  ...   
0              -0.000007  0.003004               0.000024  0.002338  ...   
1               0.000016  0.000940              -0.000021  0.000867  ...   
2              -0.000004  0.000344              -0.000007  0.000288  ...   
3               0.000006  0.000351              -0.000002  0.000294  ...   
4              -0.000012  0.006838              -0.000043  0.003785  ...   

  right armBackLateral_L_Y           right armBackLateral_L_Z            \
                      mean       std                     mean       std   
0                -0.000017  0.002554                 0.000023  0.001348   
1                -0.000023  0.005057                 0.000036  0.005517   
2                 0.000011  0.003543                 0.000019  0.003069   
3                -0.000047  0.004872                 0.000045  0.006639   
4                 0.000082  0.006276                 0.000075  0.005070   

  right armBackMedial_L_X           right armBackMedial_L_Y            \
                     mean       std                    mean       std   
0           -7.141778e-06  0.002726               -0.000022  0.002492   
1           -8.968943e-06  0.005535               -0.000025  0.004616   
2            5.224939e-07  0.003292                0.000016  0.003415   
3           -4.958312e-05  0.006967               -0.000038  0.004738   
4            3.023439e-05  0.005811                0.000077  0.006352   

  right armBackMedial_L_Z            
                     mean       std  
0                0.000021  0.001630  
1                0.000035  0.006493  
2                0.000027  0.003267  
3                0.000065  0.007665  
4                0.000052  0.006001  

[5 rows x 328 columns]

In [9]:
agg_df.shape

(1225, 328)

In [10]:
agg_df = agg_df.dropna(how='any')

In [11]:
# agg_df.isnull().sum()

In [12]:
#Separating features and label
X = agg_df.iloc[:, 2:328]
y = agg_df.iloc[:, 1]

In [13]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

In [14]:
#Feature scaling

from sklearn.preprocessing import StandardScaler

sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

**Method 1: Using LDA and Random Forest**

In [15]:
#Performing LDA

from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA

lda = LDA(n_components=60)
X_train = lda.fit_transform(X_train, y_train)
X_test = lda.transform(X_test)

c:\users\chenson\appdata\local\programs\python\python37\lib\site-packages\sklearn\discriminant_analysis.py:466: ChangedBehaviorWarning: n_components cannot be larger than min(n_features, n_classes - 1). Using min(n_features, n_classes - 1) = min(326, 50 - 1) = 49 components.
  ChangedBehaviorWarning)
c:\users\chenson\appdata\local\programs\python\python37\lib\site-packages\sklearn\discriminant_analysis.py:472: FutureWarning: In version 0.23, setting n_components > min(n_features, n_classes - 1) will raise a ValueError. You should set n_components to None (default), or a value smaller or equal to min(n_features, n_classes - 1).
  warnings.warn(future_msg, FutureWarning)
c:\users\chenson\appdata\local\programs\python\python37\lib\site-packages\sklearn\discriminant_analysis.py:388: UserWarning: Variables are collinear.
  warnings.warn("Variables are collinear.")


In [16]:
#Train a Random Forest Model

from sklearn.ensemble import RandomForestClassifier

classifier = RandomForestClassifier(max_depth=2, random_state=0)

classifier.fit(X_train, y_train)
y_pred = classifier.predict(X_test)

c:\users\chenson\appdata\local\programs\python\python37\lib\site-packages\sklearn\ensemble\forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


In [17]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score

cm = confusion_matrix(y_test, y_pred)
print(cm)
print('Accuracy = ' + str(accuracy_score(y_test, y_pred)))

[[0 0 0 ... 0 1 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 1 0]
 [0 0 0 ... 0 0 5]]
Accuracy = 0.2


**Method 2: Using LDA only**

In [18]:
import numpy as np

clf = LDA()
clf.fit(X_train, y_train)
clf.score(X_test,y_test)

0.8326530612244898

# Confusion

In [19]:
confusion = pd.DataFrame(zip(clf.predict(X_test), y_test), columns=['actual', 'predicted'])

## confirm accuracy

In [20]:
confusion['compare'] = (confusion.actual == confusion.predicted)
confusion

,actual,predicted,compare
0,Socks,Socks,True
1,Cat,Cat,True
2,Hungry,Please,False
3,Milk,Milk,True
4,Green,Green,True
...,...,...,...
240,Hot,Hot,True
241,Cost,Cost,True
242,Dog,Dog,True
243,Cost,What,False


In [21]:
confusion.compare.sum()/len(confusion)

0.8326530612244898

## Which signs have errors??

In [22]:
misclass = confusion[confusion.compare == False]

In [23]:
misclass.groupby(['actual', 'predicted'])[['predicted']].agg('count')

predicted
actual  predicted           
Bad     Bug                1
Big     Hurt               1
Brush   Hungry             1
        Where              1
Bug     Deaf               2
Candy   Work               1
Clothes Happy              1
Coat    Cold               1
Cold    Work               1
Cost    What               1
Cry     More               1
Deaf    Brush              1
        Cereal             1
Drink   Orange             1
Go      Come               1
Green   Where              1
Happy   Clothes            2
        Finish             1
Hungry  Please             1
Mom     Water              1
More    Egg                1
        Small              1
        Store              1
        What               1
Neutral Coat               1
        Cold               2
        Deaf               1
        More               1
        Red                1
        Stop               1
        Store              1
Orange  Pig                1
Pig     Water              1
Stop    Finish             1
Warm    Thanks             1
Why     Yellow             1
With    Coat               1
        Cold               1

## Crosstab

In [24]:
pd.crosstab(misclass.actual, misclass.predicted)

predicted,Brush,Bug,Cereal,Clothes,Coat,Cold,Come,Deaf,Egg,Finish,...,Red,Small,Stop,Store,Thanks,Water,What,Where,Work,Yellow
actual,,,,,,,,,,,,,,,,,,,,,
Bad,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Big,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Brush,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
Bug,0,0,0,0,0,0,0,2,0,0,...,0,0,0,0,0,0,0,0,0,0
Candy,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
Clothes,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Coat,0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Cold,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
Cost,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
